In [1]:
from pycocotools.coco import COCO

val_info = r'C:\Users\simon\Desktop\cocoapi-master\PythonAPI\test\annotations\instances_train2017.json'
val_image = r'C:\Users\simon\Desktop\cocoapi-master\PythonAPI\test\train2017'

coco = COCO(val_info)  # 导入验证集
all_ids = coco.imgs.keys()
print(len(all_ids))
person_id = coco.getCatIds(catNms=['bicycle'])
print(person_id)
person_imgs_id = coco.getImgIds(catIds=person_id)
print(len(person_imgs_id))
###
'''
loading annotations into memory...
Done (t=1.45s)
creating index...
index created!
5000  # 验证集样本总数
[1]  # 人这个类的类别id
2693  # 在验证集中，包含人这个类的图像有2693张
'''
###

loading annotations into memory...
Done (t=18.12s)
creating index...
index created!
118287
[2]
3252


'\nloading annotations into memory...\nDone (t=1.45s)\ncreating index...\nindex created!\n5000  # 验证集样本总数\n[1]  # 人这个类的类别id\n2693  # 在验证集中，包含人这个类的图像有2693张\n'

In [7]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw

# the path you want to save your results for coco to voc
savepath = "./result/"#儲存位置
img_dir = savepath + 'images/'#存圖片
anno_dir = savepath + 'Annotations/'#存label
# datasets_list=['train2014', 'val2014']
datasets_list = ['train2017']#載入資料label檔案

#classes_names = ['car', 'bicycle', 'person', 'motorcycle', 'bus', 'truck']
classes_names = ['bicycle']
# Store annotations and train2014/val2014/... in this folder
dataDir = 'C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test'#label檔案位置

headstr = """\
<annotation>
    <folder>VOC</folder>
    <filename>%s</filename>
    <source>
        <database>My Database</database>
        <annotation>COCO</annotation>
        <image>flickr</image>
        <flickrid>NULL</flickrid>
    </source>
    <owner>
        <flickrid>NULL</flickrid>
        <name>company</name>
    </owner>
    <size>
        <width>%d</width>
        <height>%d</height>
        <depth>%d</depth>
    </size>
    <segmented>0</segmented>
"""
objstr = """\
    <object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
"""

tailstr = '''\
</annotation>
'''


# if the dir is not exists,make it,else delete it
def mkr(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        os.mkdir(path)
    else:
        os.mkdir(path)


mkr(img_dir)
mkr(anno_dir)


def id2name(coco):
    classes = dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']] = cls['name']
    return classes


def write_xml(anno_path, head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr % (obj[0], obj[1], obj[2], obj[3], obj[4]))
    f.write(tail)


def save_annotations_and_imgs(coco, dataset, filename, objs):
    # eg:COCO_train2014_000000196610.jpg-->COCO_train2014_000000196610.xml
    anno_path = anno_dir + filename[:-3] + 'xml'
    img_path = dataDir + '/'+dataset + '/' + filename
    print("img_path:",img_path)
    dst_imgpath = img_dir + filename

    img = cv2.imread(img_path)
    if (img.shape[2] == 1):
        print(filename + " not a RGB image")
        return
    shutil.copy(img_path, dst_imgpath)

    head = headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(anno_path, head, objs, tail)


def showimg(coco, dataset, img, classes, cls_id, show=True):
    global dataDir
    I = Image.open('%s/%s/%s' % (dataDir, dataset, img['file_name']))
    # 通過id，得到註釋的資訊
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name = classes[ann['category_id']]
        if class_name in classes_names:
            print(class_name)
            if 'bbox' in ann:
                bbox = ann['bbox']
                xmin = int(bbox[0])
                ymin = int(bbox[1])
                xmax = int(bbox[2] + bbox[0])
                ymax = int(bbox[3] + bbox[1])
                obj = [class_name, xmin, ymin, xmax, ymax]
                objs.append(obj)
                draw = ImageDraw.Draw(I)
                draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()

    return objs


for dataset in datasets_list:
    # ./COCO/annotations/instances_train2014.json
    annFile = '{}/annotations/instances_{}.json'.format(dataDir, dataset)

    # COCO API for initializing annotated data
    coco = COCO(annFile)
    '''
    COCO 物件建立完畢後會輸出如下資訊:
    loading annotations into memory...
    Done (t=0.81s)
    creating index...
    index created!
    至此, json 指令碼解析完畢, 並且將圖片和對應的標註數據關聯起來.
    '''
    # show all classes in coco
    classes = id2name(coco)
    print(classes)
    # [1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    print(classes_ids)
    for cls in classes_names:
        # Get ID number of this class
        cls_id = coco.getCatIds(catNms=[cls])
        img_ids = coco.getImgIds(catIds=cls_id)
        print(cls, len(img_ids))
        # imgIds=img_ids[0:10]
        for imgId in tqdm(img_ids):
            img = coco.loadImgs(imgId)[0]
            filename = img['file_name']
            print("filename:",filename)
            print("dataset:",dataset)
            objs = showimg(coco, dataset, img, classes, classes_ids, show=False)
            print(objs)
            save_annotations_and_imgs(coco, dataset, filename, objs)

loading annotations into memory...
Done (t=19.89s)
creating index...


  0%|                                                                                 | 5/3252 [00:00<01:12, 45.04it/s]

index created!
{1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell pho

  1%|▍                                                                               | 18/3252 [00:00<01:00, 53.27it/s]


bicycle
[['bicycle', 178, 185, 270, 231], ['bicycle', 412, 186, 465, 233]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000253965.jpg
filename: 000000229391.jpg
dataset: train2017
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
[['bicycle', 410, 161, 440, 198], ['bicycle', 539, 118, 607, 193], ['bicycle', 245, 164, 274, 199], ['bicycle', 278, 143, 351, 195], ['bicycle', 438, 161, 474, 194], ['bicycle', 224, 160, 564, 198], ['bicycle', 334, 158, 361, 196], ['bicycle', 463, 132, 488, 194], ['bicycle', 401, 167, 418, 195], ['bicycle', 417, 150, 444, 195], ['bicycle', 351, 136, 398, 196], ['bicycle', 112, 150, 198, 365]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000229391.jpg
filename: 000000057361.jpg
dataset: train2017
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
[['bicycle', 55, 339, 74, 364], ['bicycle', 293, 269, 303, 283], ['bicycle', 142, 295, 156, 322], ['bicycle', 2

...

 000000507819.jpg
dataset: train2017
bicycle
[['bicycle', 0, 156, 25, 187]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000507819.jpg
filename: 000000229301.jpg
dataset: train2017
bicycle
[['bicycle', 370, 254, 399, 284]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000229301.jpg
filename: 000000507830.jpg
dataset: train2017
bicycle
[['bicycle', 359, 204, 578, 356]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000507830.jpg
filename: 000000507833.jpg
dataset: train2017
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
[['bicycle', 402, 345, 511, 410], ['bicycle', 400, 95, 418, 114], ['bicycle', 366, 98, 380, 116], ['bicycle', 425, 97, 447, 115], ['bicycle', 550, 94, 563, 110], ['bicycle', 358, 97, 372, 116], ['bicycle', 456, 93, 479, 111], ['bicycle', 281, 97, 300, 120]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000507833.jpg
filename: 0000001719

100%|█████████████████████████████████████████████████████████████████████████████▋| 3241/3252 [00:58<00:00, 59.43it/s]


bicycle
bicycle
bicycle
[['bicycle', 194, 184, 210, 218], ['bicycle', 127, 183, 159, 215], ['bicycle', 49, 183, 69, 238], ['bicycle', 206, 185, 217, 210], ['bicycle', 401, 178, 414, 204]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000442312.jpg
filename: 000000049097.jpg
dataset: train2017
bicycle
[['bicycle', 25, 29, 378, 312]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000049097.jpg
filename: 000000360396.jpg
dataset: train2017
bicycle
[['bicycle', 370, 1, 500, 110]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000360396.jpg
filename: 000000286678.jpg
dataset: train2017
bicycle
[['bicycle', 69, 74, 328, 258]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000286678.jpg
filename: 000000548822.jpg
dataset: train2017
bicycle
[['bicycle', 0, 107, 57, 239]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000548822.jpg
filename: 00000054

100%|██████████████████████████████████████████████████████████████████████████████| 3252/3252 [00:58<00:00, 55.17it/s]

filename: 000000081903.jpg
dataset: train2017
bicycle
[['bicycle', 105, 171, 126, 192]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000081903.jpg
filename: 000000090103.jpg
dataset: train2017
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
bicycle
[['bicycle', 0, 96, 282, 377], ['bicycle', 329, 125, 640, 356], ['bicycle', 264, 162, 346, 210], ['bicycle', 415, 222, 483, 317], ['bicycle', 432, 204, 496, 266], ['bicycle', 18, 138, 128, 241], ['bicycle', 185, 154, 282, 193], ['bicycle', 217, 133, 282, 192], ['bicycle', 316, 169, 396, 235], ['bicycle', 1, 263, 100, 427], ['bicycle', 5, 102, 468, 368]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000090103.jpg
filename: 000000262136.jpg
dataset: train2017
bicycle
[['bicycle', 162, 165, 282, 343]]
img_path: C:/Users/simon/Desktop/cocoapi-master/PythonAPI/test/train2017/000000262136.jpg
filename: 000000008187.jpg
dataset: train2017
bicycle
[['bicycle